In [1]:
# GFlowNet
from matgfn.gflow.environments.sequence import SequenceEnvironment
from matgfn.gflow.agent import TrajectoryBalanceGFlowNet
from matgfn.gflow.flow_models.lstm import LSTM

# MOF
from matgfn.reticular import PormakeStructureBuilder
import nglview as nv

# Utils
from tqdm import tqdm
import math
import subprocess
import os
import tempfile

In [ ]:
builder = PormakeStructureBuilder(topology_string="ffc", include_edges=True, block_rmsd_cutoff=0.1)
cutoff = 5000
print(f"Number of potential materials with allowed building blocks: {math.prod([len(x) for x in builder.mask.forward_actions_at_each_slot]):,}")

In [ ]:
# Make a random material
sequence = builder.random_sequence()
structure =  builder.make_structure(sequence)

# Visualise it with Pymatgen
nv.show_pymatgen(structure)

In [7]:
# Create reward using Zeo++
class SurfaceAreaRewarder():
    def __init__(self, builder, cutoff, temporary_dir = "scratch"):
        self.builder = builder
        self.cutoff = cutoff
        self.temporary_dir = temporary_dir
        os.makedirs(temporary_dir, exist_ok=True)

    def reward(self, sequence):
        with tempfile.NamedTemporaryFile(suffix=".cif", dir=self.temporary_dir, delete=True) as temp:
            mof = builder.make_pormake_mof(sequence)
            mof.write_cif(temp.name)

            command=(['./network']  + ['-sa'] + ['1.525'] + ['1.525'] + ['2000'] + [temp.name])
            subprocess.run(command,stdout = subprocess.DEVNULL)

            output_filename = temp.name.removesuffix(".cif") + ".sa"
            
            if os.path.exists(output_filename) == False:
                output_reward = 0
            else: 
                with open(output_filename) as result_file:
                    lines=[] 
                    for line in result_file:
                        lines.append(line.rstrip())

                    # Zeo++ writes empty output file for sone non-physical MOFs. Reward them as 0.
                    if len(lines)==0:
                        output_reward = 0
                    else:
                        frags=lines[0].split()
                        NASA=float(frags[17])
                        ASA=float(frags[11])

                        # Gravimetric surface area is NASA + ASA
                        area = ASA + NASA

                        output_reward = math.exp((area - cutoff) / cutoff)
            
                # Remove zeo++ output file
                # Temp file will be removed by the surrounding "with"
                os.remove(output_filename)            

            return output_reward

In [13]:
rewarder = SurfaceAreaRewarder(builder, cutoff = 5000, temporary_dir = "scratch")

In [21]:
sequence = builder.random_sequence()
print(sequence, rewarder.reward(sequence))

['N87', 'N210', 'N19', 'E9', 'E161'] 1.3528858803607724


In [22]:
env = SequenceEnvironment(
    token_vocabulary = builder.token_vocabulary, 
    termination_token = builder.termination_token,
    reward_function = lambda s: rewarder.reward(s),
    render_function = None,
    mask = builder.mask,
    max_sequence_length = builder.n_slots, min_sequence_length = builder.n_slots
    )

flow_model = LSTM(token_vocabulary = builder.token_vocabulary, n_actions = env.action_space.n)
agent = TrajectoryBalanceGFlowNet(env, flow_model)

In [ ]:
agent.train(True)
observations, infos, rewards, losses, logZs = agent.fit(learning_rate=5e-3, num_episodes=200, minibatch_size=5)

In [ ]:
agent.eval()
trained_observations = []

for i in tqdm(range(0, 10)):
    obs, info, reward, _ = agent.sample()
    sequence = agent.env._sequence[:-1]
    trained_observations.append([sequence, 0 if reward == 0 else math.log(reward) * cutoff + cutoff])

In [ ]:
trained_observations